In [2]:
## import certain packages
from math import log, sqrt, pi, exp, e
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [3]:
# PARAMETERS
# Option type option_type :: call : 'c' or put : 'p'
# Underlying price (per share): S; 
# Strike price of the option (per share): K;
# Time to maturity (in % of years): t;
# Continuously compounding risk-free interest rate: r;
# Dividend yield : q;
# One of those:
#   1. Volatility: sigma;
#   2. Price: P;

class Option():
  def __init__(self, option_type, S, K, t, r, q, **kwargs):

    def get_implied_volatility(Price):
      sigma = 0.001
      if option_type == 'c':
          while sigma < 10:
            d1_ = (log(S/K)+((r-q)+0.5*sigma**2)*t) / (sigma*sqrt(t))
            d2_ = d1_- sigma*sqrt(t)
            Price_implied = S*exp(-q*t)*norm.cdf(d1_)-K*exp(-r*t)*norm.cdf(d2_)
            if Price-(Price_implied) < 0.001:
                return round(sigma,3)
            sigma += 0.001
      if option_type == 'p':
          while sigma < 10:
            d1_ = (log(S/K)+((r-q)+0.5*sigma**2)*t) / (sigma*sqrt(t))
            d2_ = d1_- sigma*sqrt(t)
            Price_implied = K*exp(-r*t) * norm.cdf(-d2_) - ( S * exp(-q*t)) * norm.cdf(-d1_)
            if Price-(Price_implied) < 0.001:
                return round(sigma,3)
            sigma += 0.001

    # DEFINE IMPLIED VOLATILITY OR PRICE
    if 'sigma' in kwargs:
      if 'P' in kwargs:
        raise Exception('Can\'t compute price (P) and volatility (sigma) at the same time')
      else:
        self.sigma = kwargs['sigma']
        self.d1 = (log(S/K)+((r-q)+0.5*self.sigma**2)*t) / (self.sigma*sqrt(t))
        self.d2 = self.d1-self.sigma*sqrt(t)

      if option_type == 'c':
        self.P = round( ( S * exp(-q*t)) * norm.cdf(self.d1) - K*exp(-r*t) * norm.cdf(self.d2), 2 )
      if option_type == 'p':
        self.P = round( K*exp(-r*t) * norm.cdf(-self.d2) - ( S * exp(-q*t)) * norm.cdf(-self.d1), 2 )
    
    if 'P' in kwargs:
      if 'sigma' in kwargs:
        raise Exception('Can\'t compute price (P) and volatility (sigma) at the same time')
      else:
        self.P = kwargs['P']
        self.sigma = self.iv = get_implied_volatility(self.P)
        self.d1 = (log(S/K)+((r-q)+0.5*self.sigma**2)*t) / (self.sigma*sqrt(t))
        self.d2 = self.d1-self.sigma*sqrt(t)
    
    # GREEKS

    self.greeks = {
        'gamma' : (exp(-q*t) / (S*self.sigma*sqrt(t))) * norm.pdf(self.d1),
        'vega' : 0.01*(S*exp(-q*t)*norm.pdf(self.d1)*sqrt(t)),
        'vanna' : -exp(-q*t)*norm.pdf(self.d1)*self.d2/self.sigma
    }

    if option_type == 'c':
      self.greeks['delta'] = exp(-q*t) * norm.cdf(self.d1)
      self.greeks['theta'] = 1/365 * (-exp(-q*t)*(S*norm.pdf(self.d1)*self.sigma)/(2*sqrt(t)) - r*K*exp(-r*t)*norm.cdf(self.d2) + q*S*exp(-q*t)*norm.cdf(self.d1))
      self.greeks['rho'] = K*t*exp(-r*t)*norm.cdf(self.d2)
      self.greeks['epsilon'] = -S*t*exp(-q*t)*norm.cdf(self.d1)
      self.greeks['charm'] = q*exp(-q*t)*norm.cdf(self.d1) - exp(-q*t)*norm.pdf(self.d1)*(2*(r-q)*t-self.d2*self.sigma*sqrt(t))/2*t*self.sigma*sqrt(t)

    if option_type == 'p':
      self.greeks['delta'] = exp(-q*t) * ( norm.cdf(self.d1) - 1 )
      self.greeks['theta'] = 1/365 * (-exp(-q*t)*(S*norm.pdf(self.d1)*self.sigma)/(2*sqrt(t)) + r*K*exp(-r*t)*norm.cdf(-self.d2) - q*S*exp(-q*t)*norm.cdf(-self.d1))
      self.greeks['rho'] = -K*t*exp(-r*t)*norm.cdf(-self.d2)
      self.greeks['epsilon'] = S*t*exp(-q*t)*norm.cdf(-self.d1)
      self.greeks['charm'] = -q*exp(-q*t)*norm.cdf(-self.d1) - exp(-q*t)*norm.pdf(self.d1)*(2*(r-q)*t-self.d2*self.sigma*sqrt(t))/2*t*self.sigma*sqrt(t)
    
    self.greeks['lambda'] = self.greeks['delta'] * S / self.P

    self.S, self.K, self.t, self.r, self.q = S, K, t, r, q

    for greek in self.greeks.keys():
      self.greeks[greek] = round(self.greeks[greek],3)

In [9]:
disney_call = Option(option_type='c', S=52.35, K=60, t=87/365, r=0.01, q=0.018, P=0.5)
disney_call.iv

0.261

In [15]:
walmart_call = Option(option_type='c', S=52.35, K=60, t=87/365, r=0.01, q=0.018, sigma=0.261)
walmart_call.P

0.5

In [17]:
walmart_call.greeks

{'charm': 0.002,
 'delta': 0.153,
 'epsilon': -1.907,
 'gamma': 0.035,
 'lambda': 16.0,
 'rho': 1.787,
 'theta': -0.009,
 'vanna': 1.038,
 'vega': 0.06}